# Using NLP predict whether the review is positive or negative for a given dataset 
https://drive.google.com/open?id=1-TJWzdxapGhp2aElncd6RH6zOpSAf69X

In [1]:
import pandas as pd
import numpy as np
import nltk
import re # regular expression
import string
# CountVectorizer used to count words and form matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Data Gathering

In [2]:
dataset = pd.read_csv("Restaurant_Reviews.tsv", sep='\t', encoding='utf-8')
len(dataset)

1000

# Data cleaning

In [3]:
dataset.isnull().any()

Review    False
Liked     False
dtype: bool

In [4]:
dataset.duplicated().any()

True

In [5]:
dataset.drop_duplicates(keep='first', inplace=True)

In [6]:
dataset.duplicated().any()

False

In [7]:
dataset.describe()

,Liked
count,996.000000
mean,0.501004
std,0.500250
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [8]:
# corpus is a collection of text
# data cleaning steps
# 1. Remove punctuation, 2. Remove numbers and 3. Lowercase letters
porter = nltk.PorterStemmer()
corpus =[]

In [9]:
for text in dataset.Review:
    text = text.lower()  # converting text into lowercase
    text = re.sub('\[.*?\]', '', text)  # removing .,*,?,.... symbols
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # revoming punctuation
    text = re.sub('\w*\d\w*', '', text)  # removing numbers
    text = re.sub("[^a-zA-Z]", ' ', text)
    words = text.split()  # spliting sentence into words
    # stemming is the process of extracting root word from a word
    # stop words are those which express state of words like a, an, as, those, when etc.
    stem_words = [ porter.stem(word) for word in words if word not in set( nltk.corpus.stopwords.words('english') ) ]
    stemmed_sentence = ' '.join(stem_words)
    corpus.append( stemmed_sentence )

In [10]:
# re-aranging reviews with cleaned review


# Tokenization

In [11]:
# It consists three steps 
# Clean Text - remove excess, unnecessary parts of the text
# Tokenize Text - split the text into smaller pieces
# Document-Term Matrix - put into a matrix so a machine can read it

In [12]:
tokenizer = CountVectorizer( max_features = 10000 )
tokenized_data = tokenizer.fit_transform( corpus ).toarray()
document_term_matrix = pd.DataFrame( tokenized_data, columns=tokenizer.get_feature_names() )
document_term_matrix.index = dataset.index

In [13]:
document_term_matrix.head()

,absolut,absolutley,accid,accommod,accomod,accordingli,account,ach,acknowledg,across,...,yelper,yet,youd,youll,your,yucki,yukon,yum,yummi,zero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Modeling 

In [14]:
X = tokenized_data
y = dataset['Liked']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [15]:
random_forest_classifier = RandomForestClassifier(n_estimators = 300)
random_forest_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [16]:
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)

C:\Users\akash\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [17]:
support_vector_classifier = SVC()
support_vector_classifier.fit(X_train, y_train)

C:\Users\akash\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [18]:
decision_tree_classifier = DecisionTreeClassifier()
decision_tree_classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

# Prediction

In [19]:
rfc_prediction = random_forest_classifier.predict(X_test)
lr_prediction = logistic_regression.predict(X_test)
svc_prediction = support_vector_classifier.predict(X_test)
decision_tree_prediction = decision_tree_classifier.predict(X_test)

# Accuracy

In [20]:
print("Random forest:", accuracy_score(rfc_prediction, y_test)*100)
print("Logistic Regression:", accuracy_score(lr_prediction, y_test)*100)
print("Support Vector:", accuracy_score(svc_prediction, y_test)*100)
print("Decision Tree:", accuracy_score(decision_tree_prediction, y_test)*100)

Random forest: 75.25083612040135
Logistic Regression: 78.92976588628763
Support Vector: 49.49832775919732
Decision Tree: 71.57190635451505


# Confusion Matrix

In [21]:
print("Random forest:")
print(confusion_matrix(rfc_prediction, y_test))
print("Logistic Regression:")
print(confusion_matrix(lr_prediction, y_test))
print("Support Vector:")
print(confusion_matrix(svc_prediction, y_test))
print("Decision Tree:")
print(confusion_matrix(decision_tree_prediction, y_test))

Random forest:
[[134  57]
 [ 17  91]]
Logistic Regression:
[[125  37]
 [ 26 111]]
Support Vector:
[[  0   0]
 [151 148]]
Decision Tree:
[[120  54]
 [ 31  94]]


# Classification Report

In [22]:
print("Random forest:")
print(classification_report(rfc_prediction, y_test))
print("Logistic Regression:")
print(classification_report(lr_prediction, y_test))
print("Support Vector:")
print(classification_report(svc_prediction, y_test))
print("Decision Tree:")
print(classification_report(decision_tree_prediction, y_test))

Random forest:
              precision    recall  f1-score   support

           0       0.89      0.70      0.78       191
           1       0.61      0.84      0.71       108

   micro avg       0.75      0.75      0.75       299
   macro avg       0.75      0.77      0.75       299
weighted avg       0.79      0.75      0.76       299

Logistic Regression:
              precision    recall  f1-score   support

           0       0.83      0.77      0.80       162
           1       0.75      0.81      0.78       137

   micro avg       0.79      0.79      0.79       299
   macro avg       0.79      0.79      0.79       299
weighted avg       0.79      0.79      0.79       299

Support Vector:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.49      0.66       299

   micro avg       0.49      0.49      0.49       299
   macro avg       0.50      0.25      0.33       299
weighted avg       1.0

C:\Users\akash\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# Testing

In [27]:
def predict_review(text):
    predict_corpus = []
    text = text.lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub("[^a-zA-Z]", ' ', text)
    words = text.split()
    stem_words = [ porter.stem(word) for word in words if word not in set( nltk.corpus.stopwords.words('english') ) ]
    stemmed_sentence = ' '.join(stem_words)
    predict_corpus.append( stemmed_sentence )
    tokenizer = CountVectorizer(max_features = 10000)
    tokenized_data = tokenizer.fit_transform(corpus + predict_corpus).toarray()
    X = tokenized_data[-1].reshape(1, -1)
    prediction = random_forest_classifier.predict(X)
    if prediction == 1:
        return "Positive"
    else:
        return "Negative"

In [29]:
print(predict_review(input()))

food was good
Positive


In [30]:
print(predict_review(input()))

worst taste and service quality is poor
Negative
